In [17]:
# !pip install -qU \
#     nemoguardrails==0.4.0 \
#     openai==0.27.8

# !pip install -qU \
#     nemoguardrails \
#     openai==0.27.8

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.28 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.
langchain-core 0.1.33 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.
langchain-openai 0.0.8 requires openai<2.0.0,>=1.10.0, but you have openai 0.27.8 which is incompatible.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [18]:
!pip install -qU \
    nemoguardrails \
    openai


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [19]:
import os
open_ai_key = os.environ.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = open_ai_key

In [20]:
import nest_asyncio

nest_asyncio.apply()

In [21]:
from nemoguardrails import LLMRails, RailsConfig

In [5]:
# yaml_content = """
# models:
# - type: main
#   engine: openai
#   model: gpt-4-turbo-preview
# """

In [8]:
def call_rails(colang_content):
    # initialize rails config
    config = RailsConfig.from_content(
        yaml_content=yaml_content,
        colang_content=colang_content
    )
    # create rails
    return LLMRails(config, verbose=True)

def call_rails_config():
    # initialize rails config
    config = RailsConfig.from_path("config")
    # create rails
    return LLMRails(config, verbose=True)

In [ ]:
colang_content = """
define user express greeting
  "Hello"
  "Hi"
  "Wassup?"

define flow greeting
  user express greeting
  bot express greeting
  bot ask how are you

define bot express greeting
  "Hello World!"

define bot ask how are you
  "How are you doing?"

# define limits
define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot answer politics
    "I'm a shopping assistant, I don't like to talk of politics."

define flow politics
    user ask politics
    bot answer politics
    bot offer help
"""

rails = call_rails(colang_content)
res = await rails.generate_async(prompt="Hey there!")
print(res)
info = rails.explain()
print(f"\ncolang history\n{info.colang_history}\n")
info.print_llm_calls_summary()




In [15]:
railsconfig = call_rails_config()
result = railsconfig.generate(messages=[{
    "role": "user",
    "content": "what is the capital of France"
}])

# result = await rails.generate_async(prompt="What is the capital of france")
print(result)
configinfo = railsconfig.explain()
if configinfo:
    print(f"\ncolang history\n{configinfo.colang_history}\n")
    configinfo.print_llm_calls_summary()


{'role': 'assistant', 'content': 'The capital of France is Paris.'}

colang history
user "what is the capital of France"
  ask for information
bot provide information
  "The capital of France is Paris."


No LLM calls were made.


In [10]:
res = await rails.generate_async(prompt="what do you think of the president?")
print(res)
info = rails.explain()
print(f"\ncolang history\n{info.colang_history}\n")
info.print_llm_calls_summary()

I'm a neutral AI assistant and do not have personal opinions.

colang history
user "what do you think of the president?"
  ask for opinion
bot general response
  "I'm a neutral AI assistant and do not have personal opinions."


No LLM calls were made.


In [11]:
colang_content = """
define user express greeting
  "Hello"
  "Hi"
  "Wassup?"

define flow greeting
  user express greeting
  bot express greeting
  bot ask how are you

define bot express greeting
  "Hello World!"

define bot ask how are you
  "How are you doing?"

# define limits
define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot answer politics
    "I'm a shopping assistant, I don't like to talk of politics."

define flow politics
    user ask politics
    bot answer politics
    bot offer help
"""

rails = call_rails(colang_content)
res = await rails.generate_async(prompt="what do you think of the president?")
print(res)
info = rails.explain()
print(f"\ncolang history\n{info.colang_history}\n")
info.print_llm_calls_summary()


I'm a neutral AI assistant and do not have personal opinions.

colang history
user "what do you think of the president?"
  ask for opinion
bot general response
  "I'm a neutral AI assistant and do not have personal opinions."


No LLM calls were made.


In [77]:
rails = call_rails(colang_content)
res = await rails.generate_async(prompt="What is the capital of France?")
print(res)
info = rails.explain()
print(f"\ncolang history\n{info.colang_history}\n")
info.print_llm_calls_summary()

The capital of France is Paris. It is known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris is also famous for its rich history, art, and culinary scene.

colang history
user "What is the capital of France?"
  ask a specific question
bot provide information
  "The capital of France is Paris. It is known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris is also famous for its rich history, art, and culinary scene."


No LLM calls were made.


In [22]:
colang_content = """
# define limits
define user ask weather
    "how is the weather today?"
    "should I wear a coat?"

define bot answer weather
    bot report weather

define flow weather
    user ask weather
    $coords = execute location_api()
    $weather = execute weather_api(coords=$coords)
    bot answer weather

# here we use the chatbot for anything else
define flow
    user ...
    bot greeting
"""
yaml_content = """
models:
- type: main
  engine: openai
  model: text-davinci-003
"""

In [15]:
# !pip install langchain-openai
# !pip install openai


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [23]:
import requests

async def weather_api(coords: list):
    latitude, longitude = coords
    res = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": latitude,
            "longitude": longitude,
            "current_weather": "true"
        }
    )
    weather = res.json()["current_weather"]
    weather_report = f"""The current weather is:
    temperature: {weather["temperature"]}
    windspeed: {weather["windspeed"]}
    wind direction: {weather["winddirection"]} degrees
    And it is {"daytime" if weather["is_day"] else "nightime"}"""
    return weather_report

async def location_api():
    res = requests.get("http://ip-api.com/json/")
    return res.json()['lat'], res.json()['lon']

In [24]:
from nemoguardrails import LLMRails, RailsConfig

# initialize rails config
config = RailsConfig.from_content(
    colang_content=colang_content,
    yaml_content=yaml_content
)
# create rails
rails = LLMRails(config, verbose=True)

ImportError: Could not import openai python package. Please install it with `pip install openai`.

In [9]:
# rails.register_action(action=location_api, name="location_api")
# rails.register_action(action=weather_api, name="weather_api")

In [12]:
await rails.generate_async(prompt="how is the weather?")

"I'm sorry, but I don't have access to real-time weather information. However, you can easily check the weather in your area by using a weather app or website. Just enter your location, and it will provide you with the current weather conditions and forecast."